<a href="https://colab.research.google.com/github/Ifeanyi55/GephiDatasets/blob/main/KG_Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the necessary packages
!pip install -q --upgrade google
!pip install -q kg-gen

In [ ]:
import time
from google import genai
from google.colab import userdata
from kg_gen import KGGen

# set your Gemini api key in the notebook secret
api_key = userdata.get("GEMINI_API_KEY")

client = genai.Client(api_key=api_key)

prompt = "Research the history of knowledge graphs."

interaction = client.interactions.create(
    input=prompt,
    agent='deep-research-pro-preview-12-2025',
    tools=[{"type": "google_search"}],
    background=True
)

# check to ensure the research agent has started working
print(f"Research started: {interaction.id}")

while True:
    interaction = client.interactions.get(interaction.id)
    if interaction.status == "completed":
        print(interaction.outputs[-1].text) # prints out the research outcome
        break
    elif interaction.status == "failed":
        print(f"Research failed: {interaction.error}")
        break
    time.sleep(10)

# initialize the knowledge graph generator client
kg = KGGen(
    model="gemini/gemini-2.5-pro",
    temperature=0.0,
    api_key=api_key,
    )

# generate knowledge graph from the text
graph = kg.generate(input_data = interaction.outputs[-1].text)

# export and visualize knowledge graph in the browser with the explorer
KGGen.visualize(graph=graph,output_path="/content/sample_data/kgresearch.html",open_in_browser=True)

In [ ]:
# visualize with networkx
import matplotlib.pyplot as plt
import networkx as nx

# convert the graph to a networkx graph object
g = kg.to_nx(graph)

pos = nx.spring_layout(g)
nx.draw(g, pos, with_labels=True, node_size=500, font_size=8)
edge_labels = nx.get_edge_attributes(g, 'label')
nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels, font_size=6)
plt.show()

In [ ]:
# export as graphml
nx.write_graphml(g, "/content/sample_data/kg_research.graphml")